In [1]:
%pip install ultralytics
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install opencv-python

Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cu118
ERROR: Could not find a version that satisfies the requirement torchaudio (from versions: none)
ERROR: No matching distribution found for torchaudio
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# 라이브러리 임포트
import os
import json
import cv2
import torch
import shutil
from pathlib import Path
from ultralytics import YOLO

# CUDA 사용 가능 여부 확인
if torch.cuda.is_available():
   device = "cuda"
   print(f"CUDA를 사용하여 처리를 시작합니다. (GPU: {torch.cuda.get_device_name(0)})")
else:
   device = "cpu"
   print("CUDA를 사용할 수 없어 CPU로 처리를 시작합니다.")

def analyze_video_people_count(video_path, sample_interval=7, min_presence_ratio=0.3, min_size_ratio=0.1):
   """
   비디오에서 주 사용자 수만 검출
   Args:
       video_path: 비디오 파일 경로
       sample_interval: 몇 프레임마다 검사할지
       min_presence_ratio: 전체 시간 중 최소 등장 비율 (0~1)
       min_size_ratio: 화면 대비 최소 크기 비율 (0~1)
   """
   try:
       model = YOLO('yolov8s.pt').to(device)
       cap = cv2.VideoCapture(str(video_path))
       if not cap.isOpened():
           return 1

       frame_count = 0
       total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
       frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
       frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
       min_area = frame_width * frame_height * min_size_ratio

       # 각 사람별 등장 횟수 추적
       person_tracks = {}  # {track_id: count}
       
       print(f"\n{Path(video_path).name} 분석 중...")
       print(f"총 프레임 수: {total_frames}")
       
       while cap.isOpened():
           ret, frame = cap.read()
           if not ret:
               break

           frame_count += 1
           if frame_count % sample_interval != 0:
               continue

           if frame_count % (total_frames // 10) == 0:
               progress = (frame_count / total_frames) * 100
               print(f"진행률: {progress:.1f}%")

           results = model.track(frame, persist=True, classes=[0])  # tracking 모드 사용
           
           if results[0].boxes.id is not None:
               for box, track_id in zip(results[0].boxes.xywh, results[0].boxes.id):
                   # 박스 크기 계산
                   area = box[2] * box[3]  # width * height
                   track_id = int(track_id)
                   
                   # 충분히 큰 크기의 사람만 카운트
                   if area >= min_area:
                       person_tracks[track_id] = person_tracks.get(track_id, 0) + 1

       cap.release()

       # 충분한 시간 등장한 사람만 카운트
       min_frames = (total_frames / sample_interval) * min_presence_ratio
       main_people = sum(1 for count in person_tracks.values() if count >= min_frames)
       
       print(f"주 사용자 수: {main_people}명")
       return main_people

   except Exception as e:
       print(f"Error analyzing video {video_path}: {str(e)}")
       return 1

def process_folder(input_folder_path):
   """폴더 처리 메인 함수"""
   input_folder = Path(input_folder_path)
   
   # userNum이 1인 json만 필터링
   target_files = []
   print("\nJSON 파일 분석 중...")
   
   for json_file in input_folder.glob("*.json"):
       try:
           with open(json_file, 'r', encoding='utf-8') as f:
               data = json.load(f)
               if (data.get('annotations', {}).get('user_num', 0) == 1):
                   # 비디오 파일 확인 (여러 확장자 지원)
                   video_found = False
                   for ext in ['.mp4', '.avi', '.mov', '.mkv']:
                       video_file = json_file.with_suffix(ext)
                       if video_file.exists():
                           target_files.append((json_file, video_file))
                           video_found = True
                           break
                   if not video_found:
                       print(f"Warning: {json_file.name}에 대한 비디오 파일을 찾을 수 없습니다.")
       except Exception as e:
           print(f"Error processing {json_file}: {str(e)}")
           continue

   if not target_files:
       print("\n검토가 필요한 파일이 없습니다.")
       return

   print(f"\n검토 대상 파일 수: {len(target_files)}")

   need_fix_files = []
   for json_file, video_file in target_files:
       max_people = analyze_video_people_count(str(video_file))
       if max_people > 1:
           need_fix_files.append((json_file, video_file, max_people))

   # 수정이 필요한 파일이 있는 경우에만 결과 폴더 생성
   if need_fix_files:
       result_folder = input_folder.parent / f"{input_folder.name}_needFix"
       if result_folder.exists():
           shutil.rmtree(result_folder)
       result_folder.mkdir()
       print(f"\n결과 저장 폴더 생성: {result_folder}")

       # 파일 복사
       for json_file, video_file, _ in need_fix_files:
           shutil.copy2(json_file, result_folder)
           shutil.copy2(video_file, result_folder)

       print(f"\n처리 완료:")
       print(f"- 검토 대상 파일 수: {len(target_files)}")
       print(f"- 수정 필요 파일 수: {len(need_fix_files)}")
       print(f"\n수정이 필요한 파일이 {result_folder}에 복사되었습니다.")
       print("\n수정이 필요한 파일 목록:")
       for json_file, video_file, count in need_fix_files:
           print(f"- {json_file.name}: 검출된 주 사용자 수 = {count}명")
   else:
       print("\n처리 완료:")
       print(f"- 검토 대상 파일 수: {len(target_files)}")
       print("- 모든 파일의 userNum이 정상입니다.")

def get_folder_path():
   while True:
       folder_path = input("처리할 폴더 경로를 입력하세요: ").strip()
       
       # 경로에 따옴표가 있다면 제거
       folder_path = folder_path.strip('"').strip("'")
       
       if not folder_path:
           print("경로를 입력해주세요.")
           continue
           
       path = Path(folder_path)
       if not path.exists():
           print("존재하지 않는 경로입니다.")
           continue
       if not path.is_dir():
           print("폴더가 아닙니다.")
           continue
           
       return str(path)

# 메인 실행 부분
if __name__ == "__main__":
   print("YOLOv8s를 사용한 비디오 분석 프로그램")
   print("----------------------------------------")
   
   folder_path = get_folder_path()
   process_folder(folder_path)

CUDA를 사용할 수 없어 CPU로 처리를 시작합니다.
YOLOv8s를 사용한 비디오 분석 프로그램
----------------------------------------

JSON 파일 분석 중...

검토 대상 파일 수: 51

0028_1016200237.mp4 분석 중...
총 프레임 수: 1368
requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 1.0s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

Error analyzing video /Users/joyong-eun/Desktop/무제 폴더 2/28_1016/0028_1016200237.mp4: No module named 'lap'

0028_1016143929.mp4 분석 중...
총 프레임 수: 1174
requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 0.6s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

Error analyzing video /Users/joyong-eun/Desktop/무제 폴더 2/28_1016/0028_1016143929.mp4: No module named 'lap'

0028_1016103423.mp4 분석 중...
총 프레임 수: 733
requirements: Ultralytics requirem